In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным

In [4]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [5]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [6]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [7]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [10]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [12]:
W = eval_model(X_st, y, iterations=1000000, alpha=0.01)

100000 [-11.27192801  -1.45338517  -2.38300258   9.49385018] 0.25238285377547076
200000 [-16.2422902   -1.86937288  -3.90462901  13.50867797] 0.208744448215138
300000 [-20.12046178  -2.20554145  -5.06338174  16.65513544] 0.18225133394128198
400000 [-23.32957684  -2.48701096  -6.00950463  19.2580126 ] 0.16415892687401007
500000 [-26.06999299  -2.72865738  -6.81029352  21.47844134] 0.150995737106814
600000 [-28.46471856  -2.94024139  -7.50574487  23.41628013] 0.14096415738168344
700000 [-30.59526535  -3.12845003  -8.12178098  25.13788906] 0.13303809324891214
800000 [-32.51817754  -3.29802355  -8.67608723  26.68935959] 0.1265923758182375
900000 [-34.27409432  -3.4524232   -9.18120823  28.10384248] 0.12122604278612255
1000000 [-35.89309329  -3.59424249  -9.64632889  29.40588541] 0.11667083203751412


In [13]:
W = eval_model(X_st, y, iterations=10000, alpha=0.001)

1000 [ 0.25805314 -0.68199349  0.68835345  1.24114037] 0.5906912249945011
2000 [ 0.14826604 -0.69312636  0.77941081  1.21302468] 0.5686262548132578
3000 [ 0.06278003 -0.68610936  0.85154153  1.21294398] 0.5559892552843803
4000 [-0.00868429 -0.68420414  0.90686656  1.22453943] 0.5476418419000334
5000 [-0.06998375 -0.68665668  0.94958779  1.24412884] 0.5416473056405608
6000 [-0.12383979 -0.69204716  0.98295258  1.26930679] 0.5369589569591279
7000 [-0.17225684 -0.69935379  1.00933215  1.29828381] 0.5330195903831714
8000 [-0.21669501 -0.70786663  1.0304431   1.32974977] 0.5295332269569839
9000 [-0.25821006 -0.71709284  1.04752732  1.36276063] 0.5263411423015845
10000 [-0.29756287 -0.72669083  1.06148576  1.39664388] 0.5233563729128639


Теоретически можем снизить ошибку до нуля. Но пока остановимся на 10000 повторениях при альфе 0,001

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba)

In [14]:
def calc_pred_proba(W, x):
    return sigmoid(np.dot(W, x))

In [15]:
y_pred_proba = calc_pred_proba(W, X_st)
y_pred_proba

array([0.33907247, 0.44284808, 0.63408842, 0.38971253, 0.79502294,
       0.67172321, 0.80348029, 0.24402893, 0.3149916 , 0.74971083])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [16]:
def calc_pred(W, X):    
    m = X.shape[1]
    
    y_predicted = np.zeros((1, m))
    w = W.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X))
    
#     За порог отнесения к тому или иному классу примем вероятность 0.6
    for i in range(A.shape[1]):
        if (A[:,i] > 0.6): 
            y_predicted[:, i] = 1
        elif (A[:,i] <= 0.6):
            y_predicted[:, i] = 0
    
    return y_predicted

In [17]:
y_pred = calc_pred(W, X_st)
y_pred

array([[0., 0., 1., 0., 1., 1., 1., 0., 0., 1.]])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score

In [19]:
ac = y - y_pred
accuracy = np.sum([ac == 0] ) / 10
accuracy

0.8

In [21]:
k = 0
for i in range(10):
    if y[i] == 1:
        if y[i] == y_pred.T[i]:
            k += 1

TP = k / 10
TP

0.4

In [22]:
k = 0
for i in range(10):
    if y[i] == 1:
        if y[i] != y_pred.T[i]:
            k += 1

FN = k / 10
FN

0.1

In [23]:
k = 0
for i in range(10):
    if y[i] == 0:
        if y[i] != y_pred.T[i]:
            k += 1

FP = k / 10
FP

0.1

In [24]:
k = 0
for i in range(10):
    if y[i] == 0:
        if y[i] == y_pred.T[i]:
            k += 1

TN = k / 10
TN

0.4

In [25]:
precision = TP / (TP + FP)
precision

0.8

In [26]:
recall = TP / (TP + FN)
recall

0.8

In [27]:
F = 2 * precision * recall / (precision + recall)
F

0.8000000000000002